# Notebook to test ProtoNet performance

# Setup and data cleaning

## Change logs settings

In [ ]:
import pandas as pd
import numpy as np

from datasets import disable_progress_bar
disable_progress_bar() # Disable the "Map" progress bar during the tests

## Load and clean the dataset

This dataset is not on the GitHub repository.
It's composed of work experienced fetched from LinkedIn and labelled between 0 and 4 (0 if it's not related to AI and 4 if it is)

In [ ]:
dataFrame = pd.read_pickle(r'../data/7587_corrige.pkl')
subset = dataFrame[['jobTitle', 'description', 'label']].copy()

subset.reset_index(drop=True, inplace=True)
subset.replace('', np.nan, inplace=True)
subset.dropna(inplace=True)

subset['text'] = subset['jobTitle'] + ' ' + subset['description']
subset = subset[['text','label']]
subset_label_transform = subset.copy()

subset_label_transform['label'] = np.where((subset_label_transform["label"] < 3) | (subset_label_transform["label"].isna()), 0, 1)
subset_label_transform

## Split the dataset in two subsets : the training and test sets

In [ ]:
from benchmark.utility import split_dataset
train_set, test_set = split_dataset(subset_label_transform, 0.2)

## Run tests

In [ ]:
from benchmark.utility import save_to_json
from benchmark.tests import n_shot_tests, input_length_tests, language_tests, model_tests, num_epochs_tests, frozen_ratio_tests
from benchmark.train_eval_task import protonet_f1_score

### N-shots

By default SetFit uses the oversampling strategy and the Cosine Similarity loss. For instance if we have 8 positive and 8 negative examples then we have:

|   | Y | Y | Y | Y | Y | Y | Y | Y | N | N | N | N | N | N | N | N |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Y | + | + | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   | + | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   | + | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   | + | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   | + | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   | + | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   |   | + | + | - | - | - | - | - | - | - | - |
| Y |   |   |   |   |   |   |   | + | - | - | - | - | - | - | - | - |
| N |   |   |   |   |   |   |   |   | + | + | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   | + | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   | + | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   | + | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   | + | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   | + | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   |   | + | + |
| N |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | + |

- P = 2 * (8 + 7 + 6 + 5 + 4 + 3 + 2 + 1) 	= 72
- N = 8 * 8 = 64 -> + 8 duplications 		= 72
- Total = 72 + 72 = 144

In [ ]:
params = {
    "n_shot": [1, 2, 4, 6, 10],
    "n_iter": 15,
    "n_max_iter_per_shot": 10,
    "model": "sentence-transformers/paraphrase-mpnet-base-v2",
    "loss": "Cosine"
}

results, train_times, eval_times = n_shot_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/n_shot')

### Input length

In [ ]:
params = {
    "input_length_range": [[0,5],[5,25],[25,50],[50,100],[100,200],[200,350]],
	"n_shot": 10,
	"n_iter": 100,
	"model": "sentence-transformers/paraphrase-mpnet-base-v2",
	"loss": "Cosine"
}

results, train_times, eval_times = input_length_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/input_length')

### Language

In [ ]:
params = {
    "n_shot": 10,
	"lang": ['fr','en'],
	"n_iter": 100,
	"model": "sentence-transformers/paraphrase-mpnet-base-v2",
	"loss": "Cosine"
}

results, train_times, eval_times = language_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/language')

### Embedding Model

In [ ]:
params = {
    "n_shot": 10,
	"n_iter": 100,
	"loss": "Cosine",
    "model": {
        # "instructor-large":"hkunlp/instructor-large",
		"GIST-small-Embedding-v0":"avsolatorio/GIST-small-Embedding-v0",
		"gte-tiny":"TaylorAI/gte-tiny",
		# "all-mpnet-base-v2-table":"deepset/all-mpnet-base-v2-table",
  		"paraphrase-mpnet-base-v2": "sentence-transformers/paraphrase-mpnet-base-v2",
		# "all-mpnet-base-v2":"sentence-transformers/all-mpnet-base-v2",
    }
}

results, train_times, eval_times = model_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/model')

### Number of epochs

In [ ]:
params = {
    "n_shot": 10,
	"n_iter": 100,
	"model": "sentence-transformers/paraphrase-mpnet-base-v2",
	"loss": "Cosine",
    "num_epochs": [(1,0),(2,0),(4,0),(8,0),(16,0),(32,0)], # There is no classification head 
}

results, train_times, eval_times = num_epochs_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/num_epochs')

### Data sampling

Run multiple tests with different training sets but the same parameters

In [ ]:
# params = {
# 	"n_shot": 10,
# 	"n_iter": 100,
# 	"loss": "Cosine",
# 	"model": "sentence-transformers/paraphrase-mpnet-base-v2",
# 	"input_length_range":[0,9],
#     "ratio_frozen_weights": 0.5
# }

# results, train_times, eval_times = constant_params_tests(params, train_set, test_set, few_shot_model_f1_function=setfit_f1_score)

# save_to_json(results, train_times, eval_times, params,  r'../results/protonet/data_sampling')

### Frozen weights ratio

In [ ]:
params = {
    "n_shot": 10,
	"n_iter": 100,
	"model": "sentence-transformers/paraphrase-mpnet-base-v2",
	"loss": "Cosine",
    "ratio_frozen_weights": [0.1,0.3,0.5,0.7,0.9]
}

results, train_times, eval_times = frozen_ratio_tests(params, train_set, test_set, few_shot_model_f1_function=protonet_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/protonet/frozen_ratio')